# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [94]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Churn').getOrCreate()

In [95]:
df = spark.read.csv('customer_churn.csv', header=True, inferSchema=True)

In [ ]:
df.printSchema()

In [96]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [97]:
# Check for missing values
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

No missing values

In [98]:
# Check categorical column with less elements that could be used for OHE

In [99]:
from pyspark.sql.functions import col, countDistinct
df.agg(countDistinct(df['Names']).alias('Names'),
       countDistinct(df['Location']).alias('Location'),
       countDistinct(df['Company']).alias('Company')).show()

+-----+--------+-------+
|Names|Location|Company|
+-----+--------+-------+
|  899|     900|    873|
+-----+--------+-------+



### Churn is the target
Numerical column that would be used: Age, Total_purchase,Years, num_Sites, account_manager <br>

Categorical column: None will be used as too many elements. Overfitting possible. 

In [100]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### TRNASFORMING DATA FOR SPARK

In [101]:
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [102]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [103]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [104]:
df2 = assembler.transform(df)
df2.head(1)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1, features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0]))]

In [105]:
all_data = df2.select(['Churn', 'features'])
all_data.show()

+-----+--------------------+
|Churn|            features|
+-----+--------------------+
|    1|[42.0,11066.8,0.0...|
|    1|[41.0,11916.22,0....|
|    1|[38.0,12884.75,0....|
|    1|[42.0,8010.76,0.0...|
|    1|[37.0,9191.58,0.0...|
|    1|[48.0,10356.02,0....|
|    1|[44.0,11331.58,1....|
|    1|[32.0,9885.12,1.0...|
|    1|[43.0,14062.6,1.0...|
|    1|[40.0,8066.94,1.0...|
|    1|[30.0,11575.37,1....|
|    1|[45.0,8771.02,1.0...|
|    1|[45.0,8988.67,1.0...|
|    1|[40.0,8283.32,1.0...|
|    1|[41.0,6569.87,1.0...|
|    1|[38.0,10494.82,1....|
|    1|[45.0,8213.41,1.0...|
|    1|[43.0,11226.88,0....|
|    1|[53.0,5515.09,0.0...|
|    1|[46.0,8046.4,1.0,...|
+-----+--------------------+
only showing top 20 rows



## Splitting data

In [106]:
train_df, test_df = all_data.randomSplit([0.7, 0.3])

In [107]:
train_df.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                615|
|   mean|0.15934959349593497|
| stddev| 0.3662997024949379|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [108]:
test_df.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                285|
|   mean| 0.1824561403508772|
| stddev|0.38689937869106167|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



## ML models

In [109]:
from pyspark.ml.classification import LogisticRegression

In [110]:
log_Reg = LogisticRegression(featuresCol='features',
    labelCol='Churn',
    predictionCol='prediction')

In [111]:
lr_fit = log_Reg.fit(train_df)

In [112]:
train_summary = lr_fit.summary

In [113]:
train_summary.areaUnderROC

0.899735522835826

In [114]:
train_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|Churn|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|[22.0,11254.38,1....|[4.88502889523012...|[0.99249780305399...|       0.0|
|  0.0|[25.0,9672.03,0.0...|[4.62351897228850...|[0.99027727424507...|       0.0|
|  0.0|[26.0,8939.61,0.0...|[6.23626648154814...|[0.99804667399891...|       0.0|
|  0.0|[28.0,8670.98,0.0...|[7.56762623518425...|[0.99948334903094...|       0.0|
|  0.0|[28.0,11204.23,0....|[1.82797368933319...|[0.86152015857553...|       0.0|
|  0.0|[28.0,11245.38,0....|[3.76099551161602...|[0.97726818242018...|       0.0|
|  0.0|[29.0,5900.78,1.0...|[4.19003786990372...|[0.98508025879730...|       0.0|
|  0.0|[29.0,9378.24,0.0...|[4.65440915294557...|[0.99057023085786...|       0.0|
|  0.0|[29.0,9617.59,0.0...|[4.35939232459476...|[0.98737526667738...|       0.0|
|  0.0|[29.0,102

In [115]:
train_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                615|                615|
|   mean|0.15934959349593497|0.11707317073170732|
| stddev| 0.3662997024949379|0.32176916207139183|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



## Test set performance

### 1. Manually without using evaluators

In [116]:
pred = lr_fit.evaluate(test_df)

In [118]:
pred.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                285|                285|
|   mean| 0.1824561403508772|0.13333333333333333|
| stddev|0.38689937869106167|0.34053258480585086|
|    min|                  0|                0.0|
|    max|                  1|                1.0|
+-------+-------------------+-------------------+



In [119]:
pred.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|Churn|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[27.0,8628.8,1.0,...|[5.59016797025824...|[0.99627949331839...|       0.0|
|    0|[28.0,9090.43,1.0...|[1.76726239648643...|[0.85411689288244...|       0.0|
|    0|[28.0,11128.95,1....|[4.40624290728385...|[0.98794613601996...|       0.0|
|    0|[29.0,12711.15,0....|[5.35614238713679...|[0.99530308966805...|       0.0|
|    0|[30.0,6744.87,0.0...|[3.34510423192937...|[0.96594415128719...|       0.0|
|    0|[30.0,8677.28,1.0...|[4.34480361119083...|[0.98719211426917...|       0.0|
|    0|[30.0,8874.83,0.0...|[3.09437325076979...|[0.95666004956562...|       0.0|
|    0|[30.0,12788.37,0....|[2.51211455713197...|[0.92498674366126...|       0.0|
|    0|[31.0,8829.83,1.0...|[4.55448225853077...|[0.98958957170323...|       0.0|
|    0|[31.0,100

In [120]:
pred.areaUnderROC

0.9181247936612749

In [121]:
pred.weightedRecall

0.8947368421052632

In [122]:
pred.weightedPrecision

0.8887705092691243

In [123]:
pred.weightedFMeasure()

0.8880847953216374

### 2. Using evaluators

In [124]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [125]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',
    labelCol='Churn',
    metricName='areaUnderROC') 

In [126]:
# pred.prediciton contains both 'rawPrediction' and 'churn'
auc = evaluator.evaluate(pred.predictions)
auc

0.9185374711125784

## Unseen data

In [127]:
# Use all data to train new model
final_lr_model = log_Reg.fit(all_data)

In [131]:
df_new = spark.read.csv('new_customers.csv', header=True, inferSchema=True)
df_new.columns # No churn/target column here

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company']

In [132]:
df_new2 = assembler.transform(df_new)
df_new2.head(1)

[Row(Names='Andrew Mccall', Age=37.0, Total_Purchase=9935.53, Account_Manager=1, Years=7.71, Num_Sites=8.0, Onboard_date=datetime.datetime(2011, 8, 29, 18, 37, 54), Location='38612 Johnny Stravenue Nataliebury, WI 15717-8316', Company='King Ltd', features=DenseVector([37.0, 9935.53, 1.0, 7.71, 8.0]))]

In [133]:
#new_data = df_new2.select(df_new2['features'])
#new_data.show()
df_new2.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   

In [134]:
new_data2 = final_lr_model.transform(df_new2)
new_data2.select(new_data2['Names'], new_data2['prediction']).show()

+--------------+----------+
|         Names|prediction|
+--------------+----------+
| Andrew Mccall|       0.0|
|Michele Wright|       1.0|
|  Jeremy Chang|       1.0|
|Megan Ferguson|       1.0|
|  Taylor Young|       0.0|
| Jessica Drake|       1.0|
+--------------+----------+

